<h3> Create and Save a Barscan Calibration from a Sequence of Files</h3>

Most relevant function here is [calculate_barscan_calibration](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.calculate_barscan_calibration).

The output of <code>calculate_barscan_calibration</code> outputs an object containing two parts: a <b>metadata</b> (instrument name, day stamp, name of the double-detector-array), and the proper calibration data, contained in a Mantid <b>table</b> object.

In [ ]:
#
# "plot_workspace" is a utility function, which we will use a couple of times
#
%matplotlib inline
from drtsans.mono.gpsans import plot_detector
def plot_workspace(input_workspace, axes_mode='tube-pixel'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode, imshow_kwargs={})

<h4>Saving memory by transforming the events files into histograms</h4>

Although not necessary in the analysis machines, we're going to transform these events files to histogram files, thus reducing their size to about 20% the original size. We'll use these histogram files for the calibration.

This takes up quite a bit of time, so **it should be done only once**.

In [ ]:
r"""
first_run, last_run = 9905, 10016
data_files = '/HFIR/CG2/IPTS-23801/nexus/CG2_{0}.nxs.h5'
save_dir = '/HFIR/CG2/shared/sans-backend/data/new/ornl/sans/hfir/gpsans/pixel_calibration/runs_9905_10016'

import os
from mantid.simpleapi import LoadEventNexus, HFIRSANS2Wavelength, SaveNexus, DeleteWorkspaces

os.makedirs(save_dir, exist_ok=True)

print('Processing runs:')
for run in range(first_run, 1 + last_run):
    print(run, ', ', end='')

    workspace_events = 'events_{0}'.format(run)
    LoadEventNexus(Filename=data_files.format(run), LoadMonitors=False, OutputWorkspace=workspace_events)

    workspace_counts = 'counts_{0}'.format(run)
    HFIRSANS2Wavelength(InputWorkspace=workspace_events, OutputWorkspace=workspace_counts)

    file_histogram = os.path.join(save_dir, 'CG2_{0}.nxs'.format(run))
    SaveNexus(InputWorkspace='counts_{0}'.format(run), Filename=file_histogram)
    DeleteWorkspaces([workspace_events, workspace_counts])
"""

In [ ]:
import os
first_run, last_run = 9905, 10016
save_dir = '/HFIR/CG2/shared/sans-backend/data/new/ornl/sans/hfir/gpsans/pixel_calibration/runs_9905_10016'
barscan_dataset = list()  # list of histogram files
for run in range(first_run, 1 + last_run):
    file_histogram = os.path.join(save_dir, 'CG2_{0}.nxs'.format(run))
    barscan_dataset.append(file_histogram)

We can inspect a few of the bar scans by looking at the intensity pattern on a detector with default (uncalibrated) detector heights and positions

In [ ]:
from mantid.simpleapi import LoadNexus

delta = int((last_run - first_run ) / 4)
for index, run in enumerate(range(first_run, last_run, delta)):
    output_workspace = 'CG2_{0}'.format(run)
    print(output_workspace)
    LoadNexus(Filename=barscan_dataset[index * delta], OutputWorkspace=output_workspace)
    plot_workspace(output_workspace)

<h4>Calculate the barscan</h4>

We are using the **default formula** for the position of the bar:  
<code>565 - {y} - 0.0914267 * (191 - {tube})</code>  

- <code>{y}</code> is the bar position reported in the log.
- <code>{tube}</code> is the index of each tube, with index zero being the leftmost tube when viewed from the sample's position. Required if **the bar is not completely horizontal**, but it's inclined.

In [ ]:
import time
from drtsans.mono.gpsans import calculate_barscan_calibration

start_time = time.time()
formula = '565 - {y} - 0.0914267 * (191 - {tube})'
calibration = calculate_barscan_calibration(barscan_dataset, formula=formula)
print('Calibration took ', int((time.time() - start_time) / 60), 'minutes')

### Visualizing the Calibrations

We first apply the calibration to run `9960`. Then we use method [apply]() to generate a calibrated workspace `CG3_9960_calibrated` from run un calibrated run `9960`.

In [ ]:
save_dir = '/HFIR/CG2/shared/sans-backend/data/new/ornl/sans/hfir/gpsans/pixel_calibration/runs_9905_10016'
LoadNexus(Filename=os.path.join(save_dir, 'CG2_9960.nxs'), OutputWorkspace='CG2_9960')

calibration.apply('CG2_9960', output_workspace='CG2_9960_calibrated')

plot_workspace('CG2_9960_calibrated', axes_mode='xy')

Then we generate "views" of the calibrated pixels with function [as_intensities](https://scse.ornl.gov/docs/drt/sans/drtsans/pixel_calibration.html#drtsans.pixel_calibration.as_intensities). The views are two new workspaces: `views.heights` and `views.positions`. The intensities at the detector pixels of `views.heights` correspond to the heights of the detector. Thus, we can plot these "intensities" in a 2D plot of the main and wing detector array.

In [ ]:
from drtsans.mono.gpsans import as_intensities
views = as_intensities('CG2_9960_calibrated')

plot_workspace(views.positions)  # views.positions is a workspace containing pixel positions as pixel counts
plot_workspace(views.heights)  # views.heights is a workspace containing pixel positions as pixel counts

Finally, we save these views as worskpaces which can be loaded into Mantid and visualized with the Instrument Viewer

In [ ]:
from mantid.simpleapi import SaveNexus
SaveNexus(views.heights, '/tmp/pixel_heights.nxs')
SaveNexus(views.positions, '/tmp/pixel_positions.nxs')

### Analyzing the Calibrations

We first compare the middle barscan run before and after we apply the calibration

In [ ]:
plot_workspace('CG2_9960', axes_mode='xy')  # before calibration
plot_workspace('CG2_9960_calibrated', axes_mode='xy')

Two observations:

1. The vertical position of the center of mass for the detector after calibration seem to lie on positive `Y` values. That would indicate that `565` was not the correct value for the position of the bar formula='{y} - 565'.
2. After calibration, the position of the top and bottom pixels seem to increase with coordinate `X`, which may indicate a slightly tilt of the bottom edge of the bar

### Location of the Vertical Position of the Center of Mass

In [ ]:
import numpy as np
cm = 1000 * np.mean(calibration.positions)
print('Vertical position of the Center of Mass = ', cm, 'mili meters')
print('New offset = ', 565 + cm / 2, 'mili meters')

### Analyzing the possible bar tilt

We'll look at the variation in the position of the top and bottom pixels as a function of tube index. We perform a linear regression of this variation.

In [ ]:
from matplotlib import pyplot as plt
def report_tilt(pixel_positions):

    # Create a 2D array of pixel heights, dimensions are (number_tubes x pixels_in_tube)
    pixel_in_tube_count = 256
    tube_count = int(len(pixel_positions) / pixel_in_tube_count)
    positions = np.array(pixel_positions).reshape((tube_count, pixel_in_tube_count))

    def fit(tube_tip_positions):
        r"""This function will fit the bottom or top pixels against the tube index"""
        tube_indexes = np.arange(tube_count)  # heights as function of tube index
        coeffs = np.polyfit(tube_indexes, tube_tip_positions, 1)
        fitted = np.poly1d(coeffs)(tube_indexes)  # fitted positions of the tube tip
        return coeffs, fitted

    for location, tip_positions in (['top', positions[:, -1]], ['bottom', positions[:, 0]]):
        coeffs, fitted = fit(tip_positions)  # fit against tube index
        # Plot the raw positions and the fitted positions
        fig = plt.figure()
        ax = fig.add_subplot(1, 1, 1)
        ax.plot(np.arange(tube_count), tip_positions)
        ax.plot(np.arange(tube_count), fitted)
        ax.set_title(f'{location} pixels')
        # Print a few representative properties of the tilt
        print(location, ' pixels:')
        print(f'    slope = {1000 * coeffs[0]:.3f} mili-meters / tube')
        print(f'    position difference between last and first tube = {1000 * (fitted[-1] - fitted[0]):.3f} mili-meters')

report_tilt(calibration.positions)

### Removing the Bar Tilt and Centering the Detector

Thinking of the fitted positions for the bottom and top pixels, we can think of the detector array as a deformed rectangle (angles between sides different than 90 degrees), which must be transformed into a rectangle with squared angles (angles between sides equal to 90 degrees).

We take the tube in the middle of the main detector array as our reference. We will adjust every other tube so that for every tube, its top and bottom *fitted* pixel positions will coincide with the top and bottom *fitted* positions of the middle tube.

Also, since top and bottom fitted positions have a different variation with tube index, the fitted tube lenght changes sligtly with tube index. Thus, we will rescale the fitted tube length to coincide with the fitted tube length of the middle tube. This amounts to a rescaling of pixel heights.

Finally, after removing the tilt we displace the detector so that the center of mass lies at `Y=0`.

In [ ]:
from copy import deepcopy
from drtsans.pixel_calibration import Table

def untilt_and_center(a_calibration):
    # Create a 2D array of pixel heights, dimensions are (number_tubes x pixels_in_tube)
    pixel_in_tube_count = 256
    tube_count = int(len(a_calibration.positions) / pixel_in_tube_count)
    positions = np.array(a_calibration.positions).reshape((tube_count, pixel_in_tube_count))
    heights = np.array(a_calibration.heights).reshape((tube_count, pixel_in_tube_count))

    def fit(tube_tip_positions):
        r"""This function will fit the bottom or top pixels against the tube index"""
        tube_indexes = np.arange(tube_count)  # heights as function of tube index
        coeffs = np.polyfit(tube_indexes, tube_tip_positions, 1)
        fitted = np.poly1d(coeffs)(tube_indexes)  # fitted positions of the tube tip
        return coeffs, fitted

    _, fitted_top = fit(positions[:, -1])  # fitted positions of the tube tops
    _, fitted_bottom = fit(positions[:, 0])  # fitted positions of the tube bottom
    # We'll adjust the positions of the tubes to comply with the middle tube
    tube_reference_index = int(tube_count / 2)  # tube in the middle of the detector
    tube_length_reference = fitted_top[tube_reference_index] - fitted_bottom[tube_reference_index]
    # shifts_top indicate the difference in fitted positions for the tube tops with respect to the fitted positions
    # for the top of the middle tube
    shifts_top = fitted_top[tube_reference_index] - fitted_top
    shifts_bottom = fitted_bottom[tube_reference_index] - fitted_bottom
    # Calculate now the shifts for every single pixel, going tube by tube
    pixel_indexes = np.arange(pixel_in_tube_count)
    shifts = list()
    scalings = list()
    for tube_index in range(tube_count):
        a, b = shifts_bottom[tube_index], shifts_top[tube_index]
        shifts_in_tube =  a + (b - a) * pixel_indexes / pixel_in_tube_count
        shifts.append(shifts_in_tube)
        tube_length = fitted_top[tube_index] - fitted_bottom[tube_index]
        scalings_in_tube = [tube_length_reference / tube_length] * pixel_in_tube_count
        scalings.append(scalings_in_tube)

    positions_new = positions + np.array(shifts)
    heights_new = heights * np.array(scalings)

    # Set CM at y=0
    positions_new -= np.mean(positions.ravel())

    # retrieve components from the main calibration in order to construct a new calibration
    metadata = deepcopy(a_calibration.metadata)
    detector_ids = deepcopy(a_calibration.detector_ids)
    recalibration = Table(metadata,
                          detector_ids=detector_ids,
                          positions=positions_new.ravel(),
                          heights=heights_new.ravel())
    return recalibration

calibration = untilt_and_center(calibration)

Applying the adjusted calibration should create a centered detector with no dependence on tube index.

In [ ]:
calibration.apply('CG2_9960', output_workspace='CG2_9960_recalibrated')
plot_workspace('CG2_9960', axes_mode='xy')  # before calibration
plot_workspace('CG2_9960_calibrated', axes_mode='xy')  # calibrated, not adjusted
plot_workspace('CG2_9960_recalibrated', axes_mode='xy')  # calibrated and adjusted

We verify the top and bottom pixels of the recalibration do not have a dependence on tube index

In [ ]:
report_tilt(calibration.positions)

### Saving the Calibration

<h5>Calibration objects are saved to a database as two separate pieces:</h5>

- metadata (instrument name, day stamp, name of the double-detector-array) is save to a JSON file.  
- data (a table workspace) is saved to a Nexus file with SaveNexus.  

There's a default database for every instrument. The GPSANS location for the metadata JSON file:

- GPSANS: <b>/HFIR/CG2/shared/calibration/pixel_calibration.json</b>

Data tables are saved under `tables/` subdirectory:

- GPSANS: <b>/HFIR/CG2/shared/calibration/tables</b>

<h5>Calibration objects have method "<b>save</b>" to save itself to the the database. The full signature of this method:</h5>

    def save(self, database=None, tablefile=None, overwrite=False):
        r"""
        Save the calibration metadata in a JSON file, and the calibration table workspace in a Nexus file.

        Parameters
        ----------
        database: str
            Path to the JSON file where the ```metadata``` dictionary will be appended. If :py:obj:`None`,
            then the appropriate default file from ~drtsans.pixel_calibration.database_file is used.
            Currently, these are the default files:
            - BIOSANS, '/HFIR/CG3/shared/calibration/pixel_calibration.json',
            - EQSANS, '/SNS/EQSANS/shared/calibration/pixel_calibration.json',
            - GPSANS, '/HFIR/CG2/shared/calibration/pixel_calibration.json'
        tablefile: str
            Path to the Nexus file storing the pixel calibration data. If :py:obj:`None`, then
            a composite name is created using the calibration type, instrument, component,
            and daystamp. (e.g. "barscan_gpsans_detector1_20200311"). The file is saved under
            subdirectory 'tables', located within the directory of the ```database``` file.
            For instance, '/HFIR/CG3/shared/calibration/tables/barscan_gpsans_detector1_20200311.nxs'
        overwrite: bool
            Substitute existing entry with same metadata

        Raises
        ------
        ValueError
            If we save a calibration already in the database with option ```overwrite=False```.
        """

In [ ]:
# Notice we overwrite the already saved calibration, which will happen if we run this notebook more than once.
calibration.save(overwrite=True)

<h4>Load and Apply a Barscan Calibration</h4>

Most relevant function is [apply_calibrations](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.apply_calibrations)

We apply the barscan calibration to flood run 10502

In [ ]:
from mantid.simpleapi import LoadEventNexus, HFIRSANS2Wavelength
LoadEventNexus(Filename='/HFIR/CG2/IPTS-23801/nexus/CG2_10502.nxs.h5', OutputWorkspace='flood_run')
HFIRSANS2Wavelength(InputWorkspace='flood_run', OutputWorkspace='flood_run')

We apply the calibration in the database which we saved in the previous section.

<code>
def apply_calibrations(input_workspace, database=None, calibrations=[cal.name for cal in CalType],
                       output_workspace=None):

    Load and apply pixel calibrations to an input workspace.

    devs - Jose Borreguero <borreguerojm@ornl.gov>

    Parameters
    ----------
    input_workspace: str, ~mantid.api.MatrixWorkspace, ~mantid.api.IEventWorkspace
        Input workspace whose pixels are to be calibrated.
    database: str
        Path to JSON file containing metadata for different past calibrations. If :py:obj:`None`,
        the default database is used. Currently, these are the default files:
        - BIOSANS, '/HFIR/CG3/shared/calibration/pixel_calibration.json',
        - EQSANS, '/SNS/EQSANS/shared/calibration/pixel_calibration.json',
        - GPSANS, '/HFIR/CG2/shared/calibration/pixel_calibration.json'
    calibrations: str, list
        One or more of 'BARSCAN' and/or 'TUBEWIDTH'.
    output_workspace: str
         Name of the output workspace with calibrated pixels. If :py:obj:`None`, the pixels
        of the input workspace will be calibrated.
</code>

Again we plot the pixel intensities on the main detector and visually compare to the previous plot.

In [ ]:
from drtsans.mono.gpsans import apply_calibrations
apply_calibrations('flood_run', calibrations='BARSCAN', output_workspace='flood_run_calibrated')

plot_workspace('flood_run', axes_mode='xy')
plot_workspace('flood_run_calibrated', axes_mode='xy')

<h3>Calculate Tube Width Calibration</h3>

Relevant function is [calculate_apparent_tube_width](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.calculate_apparent_tube_width)

In [ ]:
import time
from mantid.simpleapi import LoadEventNexus, HFIRSANS2Wavelength
from drtsans.mono.gpsans import calculate_apparent_tube_width

#
# "plot_workspace" is a utility function, which we will use a couple of times
#
from drtsans.mono.gpsans import plot_detector
def plot_workspace(input_workspace, axes_mode='tube-pixel'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode, imshow_kwargs={})

In [ ]:
LoadEventNexus(Filename='/HFIR/CG2/IPTS-23801/nexus/CG2_10502.nxs.h5', OutputWorkspace='flood_run')
HFIRSANS2Wavelength(InputWorkspace='flood_run', OutputWorkspace='flood_workspace')

We have masked the beam center and saved the mask in a file. Here we apply the mask to the workspace using `apply_mask`

In [ ]:
from drtsans.mono.gpsans import apply_mask
mask_file = '/HFIR/CG2/shared/sans-backend/data/new/ornl/sans/hfir/gpsans/pixel_calibration/flood_files/mask_CG2_8143.xml'
apply_mask('flood_workspace', mask=mask_file)

plot_workspace('flood_workspace', axes_mode='xy')

Calculation of the apparent tube width requires that the pixel positions and heights have been calibrated with a barscan. If no good barscan is present in the database, we can use the default pixel positions and heights defined in the instrument defintion file by setting <code>load_barscan_calibration=False</code>


In [ ]:
start_time = time.time()
calibration = calculate_apparent_tube_width('flood_workspace', load_barscan_calibration=True)
print('Calibration took ', int(time.time() - start_time), 'seconds')

<h5>Calibration objects are saved to a database as two separate pieces:</h5>

- metadata (instrument name, day stamp, name of the double-detector-array) is save to a JSON file.  
- data (a table workspace) is saved to a Nexus file with SaveNexus.  

There's a default database for every instrument. The GPSANS location for the metadata JSON file:

- GPSANS: <b>/HFIR/CG2/shared/calibration/pixel_calibration.json</b>

Data tables are saved under `tables/` subdirectory:

- GPSANS: <b>/HFIR/CG2/shared/calibration/tables</b>

<h5>Calibration objects have method "<b>save</b>" to save itself to the the database. The full signature of this method:</h5>

    def save(self, database=None, tablefile=None, overwrite=False):
        r"""
        Save the calibration metadata in a JSON file, and the calibration table workspace in a Nexus file.

        Parameters
        ----------
        database: str
            Path to the JSON file where the ```metadata``` dictionary will be appended. If :py:obj:`None`,
            then the appropriate default file from ~drtsans.pixel_calibration.database_file is used.
            Currently, these are the default files:
            - BIOSANS, '/HFIR/CG3/shared/calibration/pixel_calibration.json',
            - EQSANS, '/SNS/EQSANS/shared/calibration/pixel_calibration.json',
            - GPSANS, '/HFIR/CG2/shared/calibration/pixel_calibration.json'
        tablefile: str
            Path to the Nexus file storing the pixel calibration data. If :py:obj:`None`, then
            a composite name is created using the calibration type, instrument, component,
            and daystamp. (e.g. "barscan_gpsans_detector1_20200311"). The file is saved under
            subdirectory 'tables', located within the directory of the ```database``` file.
            For instance, '/HFIR/CG3/shared/calibration/tables/barscan_gpsans_detector1_20200311.nxs'
        overwrite: bool
            Substitute existing entry with same metadata

        Raises
        ------
        ValueError
            If we save a calibration already in the database with option ```overwrite=False```.
        """

In [ ]:
# Notice we overwrite the already saved calibration, which will happen if we run this notebook more than once.
calibration.save(overwrite=True)

<h4>Load and Apply a Tube Width Calibration</h4>

In function <code>linear_density</code> below, we integrate the total intensity per tube and divide by the number of non-masked pixels in the tube, and by the tube width. Front end tubes collect more intentity than the back tubes. Similarly, front end tubes have a larger apparent tube width than back tubes. The ratio of total intensity to width should be similar for front and end tubes after the calibration.First some general imports and a couple of custom plotting functions

In [ ]:
import numpy as np
from drtsans.tubecollection import TubeCollection
from matplotlib import pyplot as plt

#
# "plot_histograms" to create fancy plots of the spectram stored in an input workspace
#
def plot_histograms(input_workspace, legend=[], xlabel='X-axis', ylabel='Y-axis', title='', linewidths=[]):
    r"""Line plot for the histograms of a workspace"""
    workspace = mtd[str(input_workspace)]
    number_histograms = workspace.getNumberHistograms()
    if len(legend) != number_histograms:
        legend = [str(i) for i in range(number_histograms)]
    if len(linewidths) != number_histograms:
        linewidths = [1] * number_histograms
    fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
    for workspace_index in range(number_histograms):
        ax.plot(workspace, wkspIndex=workspace_index, label=legend[workspace_index],
                linewidth=linewidths[workspace_index])
    ax.legend()
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.tick_params(axis='x', direction='in')
    ax.tick_params(axis='y', direction='out')
    ax.grid(True)
    fig.show()

def linear_density(workspace):
    r"""Tube total intensity per non-masked pixel and per unit length of tube width"""
    collection = TubeCollection(workspace, 'detector1').sorted(view='fbfb')
    intensities = np.array([np.sum(tube.readY) for tube in collection])
    widths = np.array([tube.width for tube in collection])
    number_pixels_not_masked = np.array([np.sum(~tube.isMasked) for tube in collection])
    return list(intensities / (number_pixels_not_masked * widths))

    We apply the barscan calibration, and the tube width calibration that we just saved

In [ ]:
from drtsans.mono.gpsans import apply_calibrations
# apply bar scan and tube width calibrations
apply_calibrations('flood_workspace', output_workspace='flood_workspace_calibrated')
plot_workspace('flood_workspace_calibrated', axes_mode='xy')

We store both linear densities in a workspace, and then we'll use matplotlib to plot both densities

In [ ]:
from mantid.simpleapi import CreateWorkspace
from mantid.api import mtd

uncalibrated_densities = linear_density('flood_workspace')
calibrated_densities = linear_density('flood_workspace_calibrated')

number_tubes = len(uncalibrated_densities)
CreateWorkspace(DataX=range(number_tubes),
                DataY=np.array([uncalibrated_densities, calibrated_densities]),
                NSpec=2,   # two histograms
                Outputworkspace='linear_densities')
plot_histograms('linear_densities',
                legend=['no calibration', 'calibrated'],
                xlabel='Tube Index', ylabel='Intensity', linewidths=[3, 1])

The oslillating intensities in the linear densities have been suppresed for most tubes, indicating the calibration is working.

<h3>Loading and Applying a Pixel Calibration (Barscan plus Tube-Width)</h3>

Relevant function is [apply_calibrations](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.apply_calibrations), which will search for <code>BARSCAN</code> and <code>TUBEWIDTH</code> calibrations appropriate to the target run

In [ ]:
import time
from mantid.simpleapi import LoadEventNexus
from drtsans.mono.gpsans import apply_calibrations

#
# "plot_workspace" is a utility function, which we will use a couple of times
#
from drtsans.mono.gpsans import plot_detector
def plot_workspace(input_workspace, axes_mode='tube-pixel'):
    return plot_detector(input_workspace, backend='mpl',axes_mode=axes_mode, imshow_kwargs={})

In [ ]:
LoadEventNexus(Filename='/HFIR/CG2/IPTS-23801/nexus/CG2_10502.nxs.h5', OutputWorkspace='workspace')
HFIRSANS2Wavelength(InputWorkspace='flood_run', OutputWorkspace='workspace')
plot_workspace('workspace', axes_mode='xy')

In [ ]:
apply_calibrations('workspace')
plot_workspace('workspace', axes_mode='xy')

We can see when the BARSCAN calibration was taken, and the time when the input data was taken. For this we use function [load_calibration](http://docs.drt-sans.ornl.gov/drtsans/pixel_calibration.html#drtsans.pixel_calibration.load_calibration) and function <code>day_stamp</code>.

In [ ]:
from drtsans.mono.gpsans import load_calibration, day_stamp
calibration = load_calibration('workspace', 'BARSCAN')
print('BARSCAN taken on', calibration.daystamp)
print('Input data taken on ', day_stamp('workspace'))